In [137]:
import pandas as pd
import datetime as dt

In [138]:
activity = pd.read_csv('../datas/activity.csv')
canact_all = pd.read_csv('../datas/canact_all_preprocessed.csv')

In [139]:
act_df = activity.sort_values(['user_id','started_at'])
act_df.head(50)

act_df.index = pd.DatetimeIndex(act_df.started_at, name='started_at').tz_convert('Asia/Tokyo')
act_df['started_jp'] = act_df.index
act_df['started_date_jp'] = pd.to_datetime(act_df['started_jp']).dt.date
act_df=act_df[(act_df['started_date_jp']>=dt.date(2021,1,12))]
# act_df['hour'] = pd.to_datetime(act_df['datetime_jp']).dt.hour
act_df.index = pd.DatetimeIndex(act_df.finished_at, name='finished_at').tz_convert('Asia/Tokyo')
act_df['finished_jp'] = act_df.index
act_df['finished_date_jp'] = pd.to_datetime(act_df['finished_jp']).dt.date
act_df=act_df[(act_df['started_date_jp']>=dt.date(2021,1,12))]

act_df = act_df.reset_index(drop=True)

In [180]:
ques_morning_dams = pd.read_csv('../datas/preprocessed_questionaire/ques_morning_dams.csv')
ques_morning_dams['datetime_jp_before3'] = pd.to_datetime(ques_morning_dams['datetime_jp_before3'])
ques_morning_dams['datetime_jp_before6'] = pd.to_datetime(ques_morning_dams['datetime_jp_before6'])
ques_morning_dams['datetime_jp_before9'] = pd.to_datetime(ques_morning_dams['datetime_jp_before9'])
ques_morning_dams['datetime_jp_before12'] = pd.to_datetime(ques_morning_dams['datetime_jp'])-dt.timedelta(hours=12)
ques_morning_dams['datetime_jp_before24'] = pd.to_datetime(ques_morning_dams['datetime_jp'])-dt.timedelta(hours=24)
ques_morning_dams['datetime_jp'] = pd.to_datetime(ques_morning_dams['datetime_jp'])

In [181]:
act_df = act_df[['user_id','activity_type_id', 'activity', 'activity_group', 'Date',
       'Time', 'am_pm','started_jp','finished_jp', 'time_spent', 
       'record_type_id', 'name', 'value', 'meal', 'working_alone',
       'hobby_break', 'housework_childcare', 'break', 'transfer',
       'web_conference', 'other', 'work_outsideduty', 'facetoface_meeting',
       'collaboration', 'remote_conference', 'started_date_jp',
       'finished_date_jp']]

ques_morning_dams = ques_morning_dams[['user_id', 'value_dams', 
       'datetime_jp', 'Date_jp', 'hour', 'positive_score', 'depressive_score',
       'anxious_score', 'datetime_jp_before3', 'datetime_jp_before6',
       'datetime_jp_before9','datetime_jp_before12','datetime_jp_before24']]

act_df2 = act_df[['user_id','activity_type_id', 'activity', 'activity_group',
       'Time', 'am_pm','started_jp','finished_jp', 'time_spent', 
       'record_type_id', 'name', 'value']]

In [182]:
# 行動データ，アンケートデータのマージ
ques_act = pd.merge(ques_morning_dams, act_df2,on='user_id')
# ques_eval.loc[ques_eval['name']=='業務に集中できていた', 'syutyu'] =ques_eval['value_eval']

# タイムゾーンを消す
ques_act['datetime_jp_before3'] = ques_act['datetime_jp_before3'].dt.tz_localize(None)
ques_act['finished_jp'] = ques_act['finished_jp'].dt.tz_localize(None)
ques_act['datetime_jp'] = ques_act['datetime_jp'].dt.tz_localize(None)
ques_act['started_jp'] = ques_act['started_jp'].dt.tz_localize(None)
ques_act['datetime_jp_before6'] = ques_act['datetime_jp_before6'].dt.tz_localize(None)
ques_act['datetime_jp_before9'] = ques_act['datetime_jp_before9'].dt.tz_localize(None)
ques_act['datetime_jp_before12'] = ques_act['datetime_jp_before12'].dt.tz_localize(None)
ques_act['datetime_jp_before24'] = ques_act['datetime_jp_before24'].dt.tz_localize(None)

In [184]:
#過去３時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before3'] = ques_act['finished_jp']-ques_act['datetime_jp_before3']
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before3'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before3']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp'])&(ques_act['datetime_jp_before3']<=ques_act['started_jp']),'act_before3'] = ques_act['datetime_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before3'])&(ques_act['datetime_jp']<=ques_act['finished_jp']),'act_before3'] = ques_act['datetime_jp']-ques_act['datetime_jp_before3']

#過去３〜６時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before3']),'act_before6'] = ques_act['finished_jp']-ques_act['datetime_jp_before6']
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before3']),'act_before6'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before6']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before3'])&(ques_act['datetime_jp_before6']<=ques_act['started_jp']),'act_before6'] = ques_act['datetime_jp_before3']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before6'])&(ques_act['datetime_jp_before3']<=ques_act['finished_jp']),'act_before6'] = ques_act['datetime_jp_before3']-ques_act['datetime_jp_before6']

#過去6〜9時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before6']),'act_before9'] = ques_act['finished_jp']-ques_act['datetime_jp_before9']
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before6']),'act_before9'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before9']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before6'])&(ques_act['datetime_jp_before9']<=ques_act['started_jp']),'act_before9'] = ques_act['datetime_jp_before6']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before9'])&(ques_act['datetime_jp_before9']<=ques_act['finished_jp']),'act_before9'] = ques_act['datetime_jp_before6']-ques_act['datetime_jp_before9']

#過去6〜9時間の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before9']),'act_before12'] = ques_act['finished_jp']-ques_act['datetime_jp_before12']
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp_before9']),'act_before12'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before12']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp_before9'])&(ques_act['datetime_jp_before12']<=ques_act['started_jp']),'act_before12'] = ques_act['datetime_jp_before9']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before12'])&(ques_act['datetime_jp_before12']<=ques_act['finished_jp']),'act_before12'] = ques_act['datetime_jp_before9']-ques_act['datetime_jp_before12']

#過去24時間分の行動時間を集計
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['finished_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before24'] = ques_act['finished_jp']-ques_act['datetime_jp_before24']
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['started_jp'])&(ques_act['finished_jp']<=ques_act['datetime_jp']),'act_before24'] = ques_act['finished_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['datetime_jp_before24']<=ques_act['started_jp'])&(ques_act['started_jp']<=ques_act['datetime_jp'])&(ques_act['datetime_jp_before24']<=ques_act['started_jp']),'act_before24'] = ques_act['datetime_jp']-ques_act['started_jp']
ques_act.loc[(ques_act['started_jp']<=ques_act['datetime_jp_before24'])&(ques_act['datetime_jp']<=ques_act['finished_jp']),'act_before24'] = ques_act['datetime_jp']-ques_act['datetime_jp_before24']

In [185]:
ques_act_user7 = ques_act.query('user_id == 7')


,user_id,value_dams,datetime_jp,Date_jp,hour,positive_score,depressive_score,anxious_score,datetime_jp_before3,datetime_jp_before6,...,finished_jp,time_spent,record_type_id,name,value,act_before3,act_before6,act_before9,act_before12,act_before24
0,7,18.0,2021-01-12 07:09:20.234,2021-01-12,7,5.666667,0.666667,1.0,2021-01-12 04:09:20.234,2021-01-12 01:09:20.234,...,2021-01-12 07:07:28.121,0 days 00:00:02.699000,NaN,NaN,NaN,0 days 00:01:54.812000,NaT,NaT,NaT,0 days 00:01:54.812000
1,7,18.0,2021-01-12 07:09:20.234,2021-01-12,7,5.666667,0.666667,1.0,2021-01-12 04:09:20.234,2021-01-12 01:09:20.234,...,2021-01-12 08:21:51.865,0 days 01:11:13.400000,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
2,7,18.0,2021-01-12 07:09:20.234,2021-01-12,7,5.666667,0.666667,1.0,2021-01-12 04:09:20.234,2021-01-12 01:09:20.234,...,2021-01-12 08:00:00.000,0 days 00:30:00,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
3,7,18.0,2021-01-12 07:09:20.234,2021-01-12,7,5.666667,0.666667,1.0,2021-01-12 04:09:20.234,2021-01-12 01:09:20.234,...,2021-01-12 12:08:26.000,0 days 03:45:38.300000,153.0,業務内容,NaN,NaT,NaT,NaT,NaT,NaT
4,7,18.0,2021-01-12 07:09:20.234,2021-01-12,7,5.666667,0.666667,1.0,2021-01-12 04:09:20.234,2021-01-12 01:09:20.234,...,2021-01-12 12:08:26.000,0 days 03:45:38.300000,154.0,業務の位置付け,NaN,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7123,7,16.0,2021-01-29 09:01:49.378,2021-01-29,9,7.000000,0.000000,0.0,2021-01-29 06:01:49.378,2021-01-29 03:01:49.378,...,2021-01-28 19:45:00.000,0 days 04:15:00,158.0,業務状況,一人,NaT,NaT,NaT,NaT,0 days 17:31:49.378000
7124,7,16.0,2021-01-29 09:01:49.378,2021-01-29,9,7.000000,0.000000,0.0,2021-01-29 06:01:49.378,2021-01-29 03:01:49.378,...,2021-01-28 19:45:00.000,0 days 04:15:00,159.0,業務環境評価,快適,NaT,NaT,NaT,NaT,0 days 17:31:49.378000
7125,7,16.0,2021-01-29 09:01:49.378,2021-01-29,9,7.000000,0.000000,0.0,2021-01-29 06:01:49.378,2021-01-29 03:01:49.378,...,2021-01-28 20:15:00.000,0 days 00:30:00,NaN,NaN,NaN,NaT,NaT,NaT,NaT,0 days 13:16:49.378000
7126,7,16.0,2021-01-29 09:01:49.378,2021-01-29,9,7.000000,0.000000,0.0,2021-01-29 06:01:49.378,2021-01-29 03:01:49.378,...,2021-01-28 21:00:00.000,0 days 00:45:00,NaN,NaN,NaN,NaT,NaT,NaT,NaT,0 days 12:46:49.378000


# 前日の行動を集計

In [211]:
ques_act_shaping = ques_act[['user_id', 'activity_group','activity_type_id','activity','value_dams', 'datetime_jp', 
       'positive_score', 'depressive_score', 'anxious_score', 'started_jp',
       'finished_jp', 'time_spent', 'record_type_id', 'name',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]

In [217]:
ques_act_shaping1 = ques_act[['user_id', 'started_jp','Date_jp',
       'finished_jp',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]
ques_act_shaping['activity'].unique()

array(['食事（業務時間外）', 'その他', '移動', '単独作業', '休憩（業務時間内）', '対面会議', '趣味・休憩',
       'Web会議', '共同作業（コミュニケーションを伴う）', '家事・育児', '業務外作業', '電話（会議）'],
      dtype=object)

In [219]:
ques_act_shaping['activity_type_id'].unique()

array([10., 12., 11.,  5.,  7.,  1.,  8.,  2.,  4.,  9.,  6.,  3.])

In [250]:
ques_act_shaping=ques_act_shaping.groupby(['user_id','started_jp','activity']).mean()
# ques_act_shaping['user_id'] = ques_act_shaping.index
finised_jp = ques_act[['user_id', 'started_jp','finished_jp',
       'act_before3', 'act_before6', 'act_before9', 'act_before12',
       'act_before24']]
ques_act_shaping2 = ques_act_shaping.reset_index()
ques_act_shaping = pd.merge(ques_act_shaping2,finised_jp,on=['user_id','started_jp'],left_on)

MergeError: Merge keys are not unique in left dataset; not a one-to-many merge

In [ ]:
ques_act_shaping

,user_id,started_jp,finished_jp,act_before3,act_before6,act_before9,act_before12,act_before24
0,7,2021-01-12 07:07:25.422,2021-01-12 07:07:28.121,0 days 00:01:54.812000,NaT,NaT,NaT,0 days 00:01:54.812000
1,7,2021-01-12 07:10:38.465,2021-01-12 08:21:51.865,NaT,NaT,NaT,NaT,NaT
2,7,2021-01-12 07:30:00.000,2021-01-12 08:00:00.000,NaT,NaT,NaT,NaT,NaT
3,7,2021-01-12 08:22:47.700,2021-01-12 12:08:26.000,NaT,NaT,NaT,NaT,NaT
4,7,2021-01-12 08:22:47.700,2021-01-12 12:08:26.000,NaT,NaT,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...
536685,88,2021-01-29 13:09:53.000,2021-01-29 17:13:09.000,NaT,NaT,NaT,NaT,NaT
536686,88,2021-01-29 13:09:53.000,2021-01-29 17:13:09.000,NaT,NaT,NaT,NaT,NaT
536687,88,2021-01-29 13:09:53.000,2021-01-29 17:13:09.000,NaT,NaT,NaT,NaT,NaT
536688,88,2021-01-29 13:09:53.000,2021-01-29 17:13:09.000,NaT,NaT,NaT,NaT,NaT
